In [1]:
import gurobipy as gp
from gurobipy import GRB
from gurobipy import *
import pandas as pd
import numpy as np


#list of teams that are included in the analysis
Team_list={
0:"Atlanta Falcons",1:"Carolina Panthers",2:"Chicago Bears",3:"Detroit Lions",
4:"Green Bay Packers",5:"Minnesota Vikings",6:"New Orleans Saints",7:"New York Giants",
8:"Philadelpia Eagles",9:"Tampa Bay Buccaneers",10:"Washington Football Team",11:"Baltimore Ravens",
12:"Buffalo Bills",13:"Cincinnati Bengals",14:"Cleveland Browns",15:"Houston Texans",
16:"Indianapolis Colts",17:"Jacksonville Jaguars",18:"Miami Dolphins",19:"New England Patroits",
20:"New York Jets",21:"Pittsburgh Steelers",22:"Tennessee Titans",23:"Dalls Cowboys"}


W=set(list(range(0,13))) #set of weeks, total of 12 weeks
T=set(list(range(0,24))) #set of teams all, total of 24 teams
#set of conference
D1=set(list(range(0,12)))  #set of AFC teams
D2=set(list(range(12,24))) #set of NFC teams

#read the distance file
E=pd.read_csv("distance.csv",index_col=0)
week = 13

In [2]:
# Create an empty model
m= gp.Model(name="NFL3")
# ADD DECISION VARIABLES HERE
x = m.addVars(T, T, W, vtype = GRB.BINARY, name = "X")

Using license file /Users/zhangmaojn/gurobi.lic
Academic license - for non-commercial use only


In [3]:
# ADD OBJECTIVE FUNCTION
# Objective is to minimize the total travel distance of all teams
obj = sum(E.iloc[i,j]*x[i,j,k] for i in T for j in T for k in W)*2

In [4]:
# ADD CONSTRAINTS
# add constraints that The season was limited to 12 weeks
m.addConstrs(sum(sum(x[i,j,k]+x[j,i,k] for k in W) for i in T) == 12 for j in T)

# add constraints that All 12 games that a team played would need to be against a different opponent
m.addConstrs(sum(x[i,j,k]+x[j,i,k]for k in W) <= 1 for i in T for j in T)
# add constraints that Each team would play at most six home games (i.e., on their home stadium)
m.addConstrs(sum(sum(x[i,j,k] for k in W) for j in T) <=6 for i in T)
# add constraints to avoid circumstances that i=i
m.addConstrs(x[i,i,k] == 0 for i in T for i in T for k in W)
# add constraints to let binary variables are bound between 0 and 1
m.addConstrs(x[i,j,k] <= 1 for i in T for j in T for k in W)
m.addConstrs(x[i,j,k] >= 0 for i in T for j in T for k in W)

# add gaming pattern constraints for 2a
m.addConstrs(sum(x[i,j,k]+x[i,j,k+1]+x[i,j,k+2] for j in T) <= 2 for i in T for k in range(week-2))
m.addConstrs(sum(x[i,j,k]+x[i,j,k+1]+x[i,j,k+2] for j in T) >= 1 for i in T for k in range(week-2))


{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (0, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,

In [5]:
#optimize the model
m.setObjective(obj, GRB.MINIMIZE)
m.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 23616 rows, 7488 columns and 97296 nonzeros
Model fingerprint: 0xed65e6fb
Variable types: 0 continuous, 7488 integer (7488 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [7e+01, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 22764 rows and 312 columns
Presolve time: 0.18s
Presolved: 852 rows, 7176 columns, 65136 nonzeros
Variable types: 0 continuous, 7176 integer (7176 binary)

Root relaxation: objective 1.659080e+05, 1556 iterations, 0.04 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    165908.00000 165908.000  0.00%     -    0s

Explored 0 nodes (2677 simplex iterations) in 0.37 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 165908 

Optimal solution found (to

In [6]:
# Print optimal values for the decision variables
print('\nDecision variables:')
for v in m.getVars():
    if v.x !=0:
        print(f'{v.varName}={v.x}, {v.obj}')


Decision variables:
X[0,1,2]=1.0, 490.0
X[0,9,12]=1.0, 912.0
X[0,15,4]=1.0, 1586.0
X[0,17,9]=1.0, 692.0
X[0,18,5]=1.0, 1300.0
X[0,22,7]=1.0, 500.0
X[1,6,10]=1.0, 1430.0
X[1,7,2]=1.0, 1250.0
X[1,8,1]=1.0, 1074.0
X[1,15,7]=1.0, 2076.0
X[1,19,12]=1.0, 1648.0
X[1,20,5]=1.0, 1250.0
X[2,3,10]=1.0, 568.0
X[2,12,4]=1.0, 1056.0
X[2,13,1]=1.0, 594.0
X[2,14,8]=1.0, 688.0
X[2,15,7]=1.0, 2164.0
X[2,23,3]=1.0, 1934.0
X[3,4,11]=1.0, 978.0
X[3,5,0]=1.0, 1382.0
X[3,12,3]=1.0, 540.0
X[3,14,5]=1.0, 340.0
X[3,16,8]=1.0, 574.0
X[3,19,8]=1.0, 1412.0
X[4,2,8]=1.0, 414.0
X[4,6,1]=1.0, 2260.0
X[4,13,4]=1.0, 1014.0
X[4,16,5]=1.0, 788.0
X[4,19,10]=1.0, 2378.0
X[4,23,0]=1.0, 2346.0
X[5,0,6]=1.0, 2226.0
X[5,2,12]=1.0, 818.0
X[5,4,9]=1.0, 558.0
X[5,14,5]=1.0, 1504.0
X[5,22,2]=1.0, 1758.0
X[5,23,8]=1.0, 1880.0
X[6,0,12]=1.0, 940.0
X[6,2,2]=1.0, 1850.0
X[6,5,8]=1.0, 2398.0
X[6,9,3]=1.0, 1314.0
X[6,15,10]=1.0, 696.0
X[6,17,6]=1.0, 1092.0
X[7,3,3]=1.0, 1226.0
X[7,8,0]=1.0, 192.0
X[7,10,8]=1.0, 446.0
X[7,18,10]=1.0, 25